In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pysr import PySRRegressor

In [2]:
df = pd.read_fwf('mass.mas03', usecols=(2, 3, 11),
                 widths=(1, 3, 5, 5, 5, 1, 3, 4, 1, 13, 11, 11,
                         9, 1, 2, 11, 9, 1, 3, 1, 12, 11, 1),
                 skiprows=41, header=None,
                 index_col=False)
df.columns = ('N', 'Z',  'avEbind')

# Extrapolated values are indicated by '#' in place of the decimal place, so
# the avEbind column won't be numeric. Coerce to float and drop these entries.
df['avEbind'] = pd.to_numeric(df['avEbind'], errors='coerce')
df = df.dropna()
df['A'] = df.N + df.Z
X = df[['N', 'Z']]
y = df[['avEbind']]

df

,N,Z,avEbind,A
0,1,1,1112.283,2
1,2,1,2827.266,3
2,1,2,2572.681,3
4,3,1,1400.351,4
5,2,2,7073.915,4
...,...,...,...,...
3010,157,101,7409.669,258
3029,154,106,7342.424,260
3034,157,104,7371.396,261
3058,156,108,7298.239,264


In [3]:
model = PySRRegressor(
    update=False,
    multithreading=True,
    niterations=40,
    binary_operators=["+", "*", "^"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "log"
        # "inv(x) = 1/x",  # Custom operator (julia syntax)
        # "odd(x) = isodd(x) ? 1.0f0 : -1.0f0"
        # extra_sympy_mappings={'special': lambda x, y: x**2 + y},
    ],
    model_selection="best", # "accuracy"
    loss="loss(x, y) = (x - y)^2",  # Custom loss function (julia syntax)
)

model.fit(X, y)
print(model)

/root/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pysr/sr.py:1113: UserWarning: Resetting variable_names from X.columns
  warnings.warn("Resetting variable_names from X.columns")
  Activating project at `~/.julia/environments/pysr-0.8.7`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pysr-0.8.7/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.8.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pysr-0.8.7/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.8.7/Manifest.toml`


Started!

Cycles per second: 1.250e+02
Head worker occupation: 18.2%
Progress: 1 / 600 total iterations (0.167%)
Hall of Fame:
-----------------------------------------
Complexity  loss       Score     Equation
1           6.575e+07  -4.953e+00  -0.18784586
3           6.332e+07  1.887e-02  (N + N)
5           2.771e+07  4.131e-01  (N + (Z * Z))
7           2.728e+07  7.861e-03  ((Z * Z) + (N + N))
9           2.686e+07  7.712e-03  ((N + (Z * Z)) + (N + N))
11          2.646e+07  7.554e-03  (((N + N) + (Z * Z)) + (N + N))
13          2.607e+07  7.386e-03  (((N + N) + (N + (Z * Z))) + (N + N))
14          2.537e+07  2.729e-02  (((log_abs(N) * N) + (N + (Z * Z))) + (N + N))
16          2.355e+07  3.713e-02  (((log_abs(N) * (N + (N + N))) + (Z * Z)) + (N + N))
18          2.334e+07  4.522e-03  (((N + (log_abs(N) * (N + (N + N)))) + (Z * Z)) + (N + N))
20          2.315e+07  4.244e-03  ((((N + N) + (log_abs(N) * (N + (N + N)))) + (Z * Z)) + (N + N))

Press 'q' and then <enter> to stop exec

TypeError: unsupported operand type(s) for *: 'function' and 'Float'

In [ ]:
plt.scatter(y[:, 0], model(X)[:, 0])
plt.xlabel('Truth')
plt.ylabel('Prediction')
plt.show()


In [ ]:
pdf=df.drop(['avEbind','A', 'parity', 'LDM', 'res2'], axis = 1)
tp=pdf.set_index(['Z', 'N']).res.unstack(0)

fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
z_min=pdf.res.min()
z_max=pdf.res.max()
print(z_min, z_max)

# c = ax.pcolormesh(tp, cmap= cm.coolwarm, norm=colors.LogNorm(vmin=z_min, vmax=z_max)) 
c=ax.pcolormesh(tp, norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=z_min, vmax=z_max, base=10),
                       cmap=cm.coolwarm, shading='auto')

ax.set_title('optimized')
# set the limits of the plot to the limits of the data
ax.axis([pdf.Z.min(), pdf.Z.max(), pdf.N.min(), pdf.N.max()])
fig.colorbar(c, ax=ax)

ax.set_xlabel('Z')
ax.set_ylabel('N')

plt.show()
plt.savefig("LDM.png", dpi=150)